# Modeling Diffusion

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vis import styleIterator
from space import Location, Field, OddField
from drunks import UsualDrunk, ColdDrunk, EWDrunk
import uuid

The code up to this point, only keeps track of teh current location of the drunk. What if we were interested in the path a drunk takes? Then we need to have something like a list that we continually update. An immediate question we need to address is, "Where should we keep the list? Should it be part of the `Field` class or one of the `Drunk` classes. 

In [ ]:
import numbers
import math
import random
import seaborn as sns


#### Create a `DiffusionField` class to approximate diffusion behavior

In [ ]:

from space import Location, Field
class DiffusionField(Field):
    def __init__(self):
        super(DiffusionField, self).__init__()
        self.__occupied = set([])

    def isOccupied(self, loc):
        return loc in self.__occupied
    
    def moveDrunk(self, drunk):
        if not self.hasDrunk(drunk):
            raise ValueError("Drunk not in field")
        xDist, yDist = drunk.takeStep()
        currentLocation = self.getLoc(drunk)
        newLocation = currentLocation.move(xDist, yDist)

        if newLocation in self.__occupied:
            self.placeDrunk(drunk, currentLocation)
        else:
            self.__occupied.remove(currentLocation)
            self.__occupied.add(newLocation)
            #print("new location")
            self.placeDrunk(drunk, newLocation)
            
    def addDrunk(self, drunk, loc):
        
        if loc in self.__occupied:
            raise ValueError("occupied space")
        if not isinstance(loc, Location):
            raise TypeError("loc must be an instance of location")
        if self.hasDrunk(drunk):
            raise ValueError("Duplicate drunk")
                            
        else:
            self.__occupied.add(loc)
            self.placeDrunk(drunk, loc)            
    
        

In [ ]:
def getLoc(f, xr, yr):
    loc = Location(random.randint(-xr, xr), random.randint(-yr, yr))
    if not f.isOccupied(loc):
        return loc
    else:
        return get_loc(f, xr, yr)
    
def populatField(f, dtype, numDrunks, xRange, yRange):
    for i in range(numDrunks):
        d = dtype(name=str(uuid.uuid1().int))
        f.addDrunk(d, getLoc(f, xRange, yRange))

    return len(f.drunks())

In [ ]:
def viewField(f, xrange, yrange):
    xvals, yvals = [], []
    for d in f.drunks():
        loc = f.getLoc(d)
        xvals.append(loc.x)
        yvals.append(loc.y)

    plt.plot(xvals, yvals, "ro")
    plt.xlim(-xrange, xrange)
    plt.ylim(-yrange, yrange)

In [ ]:

def run_field(f, numSteps):
    for i in range(numSteps):
        for d in f.drunks():
            f.moveDrunk(d)
    return None

## Run a diffusion process

In [ ]:
f = DiffusionField()
populatField(f, UsualDrunk, 200, 200, 200)
viewField(f, 800, 800)

tmp = %timeit -o -r 1 run_field(f, 100000)

viewField(f, 800, 800)


In [ ]:

times = {}
for n in [1000, 10000, 100000]:
    f = DiffusionField()
    populatField(f, UsualDrunk, 100, 200, 200)
    tmp = %timeit -o -r 1 run_field(f, n)
    times[n] = tmp


In [ ]:
pops = {}
for n in [100, 200, 400]:
    f = DiffusionField()
    populatField(f, UsualDrunk, n, 200, 200)
    tmp = %timeit -o -r 1 run_field(f, 1000)
    pops[n] = tmp


In [ ]:
viewField(f, 800, 800)